# Guidelines for prompting

Goal of this notebook is practicing and understanding prompting principles.

Examples sourced from [Deeplearning.ai ChatGPT Prompt Engineering for Developers](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/2/guidelines) course.

LLM Model: [dolly-v2-3b](https://huggingface.co/databricks/dolly-v2-3b)

In [94]:
!export PYTORCH_ENABLE_MPS_FALLBACK=1
import time

import torch
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# print(torch.backends.mps.is_available())
# print(torch.backends.mps.is_built())
# print(torch.cuda.is_available())

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [41]:
generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map="auto", return_full_text=True)

simple_prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}")

prompt_with_context = PromptTemplate(
    input_variables=["instruction", "context"],
    template="{instruction}\n\nInput:\n{context}")

hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

llm_chain = LLMChain(llm=hf_pipeline, prompt=simple_prompt)
llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context)


def get_response(input_prompt, context=None):
    if context is None:
        return llm_chain.predict(instruction=input_prompt).lstrip()
    else:
        return llm_context_chain.predict(instruction=input_prompt, context=context).lstrip()


def reply_prompt(input_prompt, context=None):
    start = time.time()
    response = get_response(input_prompt, context)
    end = time.time()
    duration_seconds = end - start
    print(response)
    print(f"\nProcessing duration: {duration_seconds:0.2f} seconds")


In [42]:
text = f"""
What is the capital of United Kingdom?
"""
prompt = f"""
Reply the question delimited by <> in a single sentence.
<{text}>
"""
reply_prompt(prompt)

The capital of United Kingdom is London.

Processing duration: 35.33 seconds


## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

### Principle 1: Write clear and specific instructions

#### Tactics

##### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [47]:
text = f"""
You should express what you want a model to do by providing instructions that are as clear and specific as you can possibly make them. This will guide the model towards the desired output, and reduce the chances of receiving irrelevant or incorrect responses. Don't confuse writing a clear prompt with writing a short prompt. In many cases, longer prompts provide more clarity and context for the model, which can lead to more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks using few words.
```{text}```
"""
reply_prompt(prompt)

clear and specific

Processing duration: 81.23 seconds


##### Tactic 2: Ask for a structured output
- JSON, HTML

In [45]:
prompt = f"""
Generate a list of three made-up book titles along with their authors and genres.
Provide them as a list in JSON format with the following keys: book_id, title, author, genre.
"""
reply_prompt(prompt)

Here is a list of three book titles and their authors and genres:
{
  "book_id": 1,
  "title": "Harry Potter and the Methods of Rhythm",
  "author": "JK Rowling",
  "genre": "Fantasy"
},
{
  "book_id": 2,
  "title": "The Joy of Telegraph",
  "author": "Telegraph",
  "genre": " humour"
},
{
  "book_id": 3,
  "title": "Silence is Death",
  "author": "Shirley Conte",
  "genre": " Horror"
}

Processing duration: 136.93 seconds


##### Tactic 3: Ask the model to check whether conditions are satisfied

In [52]:
text = f"""
Making a cup of tea is easy! First, you need to get some water boiling. While that's happening, grab a cup and put a tea bag in it. Once the water is hot enough, just pour it over the tea bag. Let it sit for a bit so the tea can steep. After a few minutes, take out the tea bag. If you like, you can add some sugar or milk to taste. And that's it! You've got yourself a delicious cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by <>. 
If it contains a sequence of steps, re-write those steps in the following format:

Step 1 - ...
Step 2 - ...
Step 3 - ...
...
Step N - ...

If the text does not contain a sequence of steps, write nothing."

<{text}>
"""

reply_prompt(prompt)

Step 1 - Grab some water boiling
Step 2 - while that's happening, grab a cup and put a tea bag in it
Step 3 - pour it over the tea bag
Step 4 - let it sit for a bit so the tea can steep
Step 5 - after a few minutes, take out the tea bag
Step 6 - if you like, you can add some sugar or milk to taste

Processing duration: 176.14 seconds


##### Tactic 4: "Few-shot" prompting

In [56]:
prompt = f"""
You will receive a dialog between two characters delimited by triple backticks.
Your task is to explain the concept asked by <child> following the style of <grandparent>.

```
<child>: Teach me about patience.

<grandparent>: The river that carves the deepest valley flows from a modest spring; the grandest symphony originates from a single note; the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
```

"""
reply_prompt(prompt)

Patience is a virtue. Resilience is a trait you acquire through hard work. Both are essential to human nature. To better understand patience, consider the paradox: Perseverance is often harder than the initial attempt. Patience allows you to wait for what's to come; it also allows you to be comfortable with uncertainty. The more you practice patience, the easier it becomes. Resilience, on the other hand, is something that can be cultivated. Consider also: In sports, the best athletes always learn from their failures, and never quit when things get difficult. Resilience is a hard-willed quality, and requires you to be unafraid of failure. To build resilience, you have to put things into perspective and not get too caught up in the moment. In summary, patience is a virtue you acquire through experience, and resilience is a hard-willed quality that can be cultivated.

Processing duration: 198.46 seconds


### Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [88]:
text = f"""
In a charming village, siblings Jack and Jill set out on a quest to fetch water from a hilltop well. As they climbed, singing joyfully, misfortune struck—Jack tripped on a stone and tumbled down the hill, with Jill following suit. Though slightly battered, the pair returned home to comforting embraces. Despite the mishap, their adventurous spirits remained undimmed, and they continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by <> with at most 100 characters.
2 - List each name from the text.
3 - Output a JSON object that contains the following keys: summary, num_names.

Separate your answers with line breaks.

Text:
<{text}>
"""

reply_prompt(prompt_1)


{
  "summary":"In a charming village, siblings Jack and Jill set out on a quest to fetch water from a hilltop well.",
  "num_names": "2"
}

Processing duration: 139.74 seconds


##### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [87]:
prompt = f"""
Your task is to determine if the student's solution is correct or not.

To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution and evaluate if the student's solution is correct or not.

Don't decide if the student's solution is correct until you have done the problem yourself.

Question: 'Is the square root of 9 an odd number?'

Student's solution: 'No square root of 9 is an even number.'

Comparison of both solutions: ...
"""
reply_prompt(prompt)

No, the square root of 9 is not an even number.

Processing duration: 97.51 seconds
